# Microarray Analysis

In [62]:
# Extracting Microarray image data
from PIL import Image
#from Images import imageToPixmapRGB
from numpy import array, dstack, transpose, uint8, zeros

from numpy import dot, log, sqrt

# image to pixmap method from Images.py
def imageToPixmapRGB(img):
  
    img2 = img.convert('RGB')
    w, h = img2.size  
    data = img2.getdata()

    pixmap = array(data, float)
    pixmap = pixmap.reshape((h,w,3))
  
    return pixmap

def loadArrayImage(fileName, sampleName, nRows, nCols=None):
    
    if not nCols:
        nCols = nRows
    # empty matrix for data
    dataMatrix = zeros((3, nRows, nCols), float)
    
    #read in microarray image and conver to RGB 
    img = Image.open(fileName)
    pixmap = imageToPixmapRGB(img)
    
    #read size of the pixmap
    height, width, depth = pixmap.shape
    
    dx = width/nCols
    dy = height/nRows
    xSize = 1 + (width-1)//nCols
    ySize = 1 + (height-1)//nRows
    
    #finds pixel location in rows
    for row in range(nRows):
        yStart = int(row*dy)
        yEnd   = yStart + ySize
    
        # finds pixel loaction in columns
        for col in range(nCols):
            xStart = int(col*dx)
            xEnd   = xStart + xSize
        
            elementData = pixmap[yStart:yEnd,xStart:xEnd]
            dataMatrix[:, row, col] = elementData.sum(axis=(0,1))
    
    # creates microarray object with the loaded datamatrix and the samnpleName
    return Microarray(sampleName, dataMatrix)
    
        

In [63]:
# distance methods from SeqVariations.py from the sample code
# adding here as support functions for the clustering methods in microarray class

def getJoinPair(distMatrix):

    n = len(distMatrix)

    minQ = None
    joinPair = None

    for i in range(n-1):
        sumRow = sum(distMatrix[i])
  
    for j in range(i+1, n):
        sumCol = sum(distMatrix[j])
    
    dist = distMatrix[i][j]
    q = (n-2)*dist - sumRow - sumCol
      
    if (minQ is None) or (q < minQ):
        minQ = q
        joinPair = [i,j]
    
    return joinPair


def getDistToJunction(distMatrix, i, j):
  
    n = len(distMatrix)
    row = distMatrix[i]
    column = distMatrix[j]
 
    dist = distMatrix[i][j] + (sum(row)-sum(column))/(n-2)
    dist *= 0.5

    return dist


def neighbourJoinTree(distMatrix):
 
    joinOrder = []
    n = len(distMatrix)
    tree = list(range(n))  # do not need list() in Python 2
  
    while n > 2:

        x, y = getJoinPair(distMatrix)

        node = (tree[x], tree[y])
        joinOrder.append(node)
        tree.append(node)

        del tree[y]
        del tree[x]

        distX = getDistToJunction(distMatrix, x, y)
        distY = getDistToJunction(distMatrix, y, x)

        distMatrix.append([0] * (n+1))

        for i in range(n):
            if i not in (x,y):

                dist = (distMatrix[x][i]-distX) + (distMatrix[y][i]-distY)
                dist *= 0.5

                distMatrix[i].append(dist)
                distMatrix[n][i] = dist

        del distMatrix[y]
        del distMatrix[x]
  
        for row in distMatrix:
            del row[y]
            del row[x]

        n -= 1

    tree = tuple(tree)
    joinOrder.append(tree)
  
    return tree, joinOrder


In [64]:
# Microarray Class

class Microarray(object):
    
    # contstructor 
    def __init__(self, name, data, rowData=None, colData=None):
        
        self.name = name
        data = array(data)
        
        shape = data.shape
        
        if len(shape) == 3:
            self.nChannels, self.nRows, self.nCols = shape
            
        elif len(shape) == 2:
            self.nRows, self.nCols = shape
            self.nChannels = 1
            data.reshape((1, self.nRows, self.nCols))
            
        else: 
            raise Exception('Array data must have either 2 or 3 axes.')
            
        self.data = data
        self.origData = array(data)
        
        self.rowData = rowData or range(self.nRows)
        self.colData = colData or range(self.nCols)
    
    
    def resetData(self):
        
        self.data = array(self.origData)
        self.nChannels = len(self.data)
        
        
    # writes image data fromn pixmap to a text file 
    def writeData(self, fileName, separator=''):
        
        fileObj = open(fileName, 'w')
        
        for i in range(self.nRows):
            rowName = str(self.rowData[i])
            
            for j in range(self.nCols):
                colName = str(self.colData[j])
                
                values = self.data[:,i,j]
                
                lineData = [rowName, colName]
                lineData += ['%.3f' % (v,) for v in values]
                
                line = separator.join(lineData)
                fileObj.write(line + '\n')

                
    # creates image representing microarray data   
    def makeImage(self, squareSize=20, channels=None):
        
        minVal = self.data.min()
        maxVal = self.data.max()
        dataRange = maxVal - minVal
        
        adjData = (self.data - minVal) * 255 / dataRange
        adjData = array(adjData, uint8)
        
        if not channels:
            if self.nChannels ==1:
                channels = (0,0,0) # grayscale
                
            else: 
                channels = list(range(self.nChannels))[:3]
                
        pixmap = []
        
        for i in channels:
            if i is None:
                pixmap.append(zeros((self.nRows, self.nCols), uint8))
            else:
                pixmap.append(adjData[i])
        
        while len(pixmap) < 3:
            pixmap.append(zeros((self.nRows, self.nCols), uint8))
            
        pixmap = dstack(pixmap)
        img = Image.fromarray(pixmap, 'RGB')
        
        width = self.nCols * squareSize
        height = self.nRows * squareSize
        img = img.resize ((width, height))
        
        return img
    
        
    # set lower limit
    def clipBaseline(self, threshold=None, channels=None, defaultProp=0.2):
        
        if not channels:
            channels = range(self.nChannels)
        
        channels = [tuple(channels)]
        
        
        maxVal = self.data[channels].max()
        if threshold is None:
            limit = maxVal * defaultProp
        else:
            limit = threshold
        
        boolArray = self.data[channels] < limit
        indices = boolArray.nonzero()
        
        self.data[indices] = limit
        
        self.data[channels] -= limit
        self.data[channels] *= maxVal / (maxVal-limit)
        
    # value normalization methods 
    
    def normaliseSd(self, scale=1.0):
        for i in range(self.nChannels):
            self.data[i] = self.data[i] * scale / self.data[i].std()
    
    def normaliseMean(self, scale=1.0):
        for i in range(self.nChannels):
            self.data[i] = self.data[i] * scale / self.data[i].mean()
    
    def centerMean(self):
        for i in range(self.nChannels):
            self.data[i] -= self.data[i].mean()

    def normailzeZscore(self):
        self.centerMean()
        self.normaliseSd()
        
    
    def normaliseMax(self, scale=1.0, perChannel=True):
        
        if perChannel:
            for i in range(self.nChannels):
                self.data[i] = self.data[i] * scale / self.data[i].max()
        else:
            self.data = self.data * scale / self.data.max()
    
    
    def normaliseRowMax(self, scale=1.0):
        self.data[i] = self.data[i] * scale / self.data[i].max(axis=1)[:,None]
        
        
    def normaliseRowMean(self, scale=1.0):
        for i in range(self.nChannels):
            self.data[i] = self.data[i] * scale / self.data[i].mean(axis=1)[:,None]
            
            
    def normaliseColMax(self, scale=1.0):
        for i in range(self.nChannels):
            self.data[i] = self.data[i] * scale / self.data[i].max(axis=0)
    
    
    def normaliseColMean(self, scale=1.0):
        for i in range(self.nChannels):
            self.data[i] = self.data[i] * scale / self.data[i].mean(axis=0)
            
    
    def normaliseRefs(self, rows, cols, scale=1.0, channels=None):
        
        if not channels:
            channels = range(self.nChannels)
            
        channels = tuple(channels)
        refValues = self.data[channels, rows, cols]
        
        for i in channels:
            self.data[i] = self.data[i] * scale / refValues[i].mean()
    
    def normaliseLogMean(self):
        
        self.clipBaseline(threshold=0.0)
        
        for i in range(self.nChannels):
            self.data[i] = log( 1.0 + self.data[i] / self.data[i].mean()) 
            
            
    
    def normaliseQuantile(self, refData, channel=0):
        
        values = self.data[channel].flatten()
        order = values.argsort()
        
        refValues = refData.flatten()
        refValues.sort()
        
        refSelection = order.argsort()
        values = refValues[refSelection]
        
        self.data[channel] = values.reshape((self.nRows, self.nCols))
        
    
    def normaliseRowQuantile(self, channel=0):
        
        channelData = self.data[channel]
        orders = channelData.argsort(axis=1)
        sortedRows = array(channelData)
        sortedRows.sort(axis=1)
        refValues = sortedRows.mean(axis=0) # average over columns
        
        rows = range(self.nRows)
        self.data[channel,rows,:] = refValues[orders[rows,:].argsort()]
        
    
    # methods for changing array channels
    
    def checkDataSize(self, channelData): # for making sure the array size can be loaded
    
        channelData = array(channelData)

        if channelData.shape != (self.nRows, self.nCols):
            msg = 'Attempt to use data of wrong size'
            raise Exception(msg)

        return channelData

    
    def setChannel(self, channelData, index=0):
        
        channelData = self.checkDataSize(channelData)
        self.data[index] = channelData
    
    
    def addChannel(self, channelData):
        
        from numpy import append
        channelData = self.checkDataSize(channelData)
        
        self.data = append(self.data, channelData, axis=0)
        self.nChannels += 1
        
    
    def swapChannels(self, indexA, indexB):
        
        self.data[(indexB, indexA)] = self.data[(indexA, indexB)]
        
    
    def removeChannel(self, index):
        
        from numpy import delete
        
        if index < self.nChannels:
            self.data = delete(self.data, index, axis=0)
            self.nChannels -= 1

    def combineChannels(self, indexA, indexB, combFunc=None, replace=None):
       
        if not combFunc:
            import operator
            combFunc= operator.add
            
        channelData = combFunc(self.data[indexA], self.data[indexB])
        
        if replace is None:
            self.addChannel(channelData)
        else:
            self.setChannel(channelData, replace)
            
    
    def __hierarchicalRowCluster(self, dataMatrix):
        #from SeqVariation import neighbourJoinTree
        n = len(dataMatrix[0])
        distanceMatrix = zeros((n, n), float)
        
        for channelData in dataMatrix:
            for i, row in enumerate(channelData):
                diffs = channelData - row
                sqDiffs = diffs * diffs
                sqDists = sqDiffs.sum(axis=1)
                distanceMatrix[i,:] += sqDists
                
        # neighbourJoinTree is used here to create a hierarchical tree
        tree, joinOrder = neighbourJoinTree(distanceMatrix.tolist())
        
        rowOrder = list(tree)
        
        i=0
        while i < len(rowOrder):
            while not isinstance(rowOrder[i], int):
                rowOrder[i:i+1] = rowOrder[i]
            i += 1
        
        return rowOrder
    
    
    def hierarchicalCluster(self):
        
        rows = self.__hierarchicalRowCluster(self.data)
        swapped = transpose(self.data, axes=(0,2,1))
        cols = self.__hierarchicalRowCluster(swapped)
        
        data = self.data[:,rows] # Rearrange
        data = data[:,:,cols]
        
        # data = array(data.tolist()) # to fix PIL.Image bug
        
        name = self.name + '-Sorted'
        rowData = [self.rowData[i] for i in rows]
        colData = [self.colData[j] for j in cols]
        sortedArray = Microarray(name, data, rowData, colData)
        
        return sortedArray
    
    # for part three of the homework...
    def distanceMatrix(self):
        
        dataMatrix = self.data
        n = len(dataMatrix[0])
        distanceMatrix = zeros((n, n), float)

        for channelData in dataMatrix:
            for i, row in enumerate(channelData):
                diffs = channelData - row
                sqDiffs = diffs * diffs
                sqDists = sqDiffs.sum(axis=1)
                distanceMatrix[i,:] += sqDists

        return distanceMatrix


In [53]:
# TEXTBOOK EXAMPLES #

## load example image and create a microarray object
#imgFile = 'RedGreenArray.png'
#rgArray = loadArrayImage(imgFile, 'TwoChannel', 18, 17)

## writing array data to a text file example:
# rgArray.writeData('RedGreenArray.txt')
# rgArray.makeImage(20).show()

## Quantile normalisation example:
#rgArray.normaliseQuantile(rgArray.data[1], 0)
#rgArray.makeImage(25).show()

## combining channels example:
## Red and green added to channel 2
#rgArray.combineChannels(0, 1, replace=2)
## Display channel 2 as yellow
#rgArray.makeImage(20, channels=(2, 2, None)).show()

## clustering example:
#sortedArray = rgArray.hierarchicalCluster()
#sortedArray.makeImage(20).show()
#print(rgArray.rowData)
#print(sortedArray.rowData)



### Python code for importing two array channels, taking one away from the other and storing the results of positive or negative values in red or green color channels

In [54]:
# uses the example png file given it is in the same directory 

imgFile = 'RedGreenArray.png'
rgArray = loadArrayImage(imgFile, 'TwoChannel', 18, 17)

# make comparison
diff = rgArray.data[0]-rgArray.data[1]

rgArray.setChannel(diff, 0)
# flips sign of green channel
rgArray.setChannel(-diff, 1)

# clip values and set a lower threshold
rgArray.clipBaseline(threshold=0.0, channels=(0,1))

rgArray.makeImage(20).show()


/var/folders/6v/n7k6518972x6bfvr4s25jp240000gn/T/ipykernel_5096/1296736070.py:104: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  maxVal = self.data[channels].max()
/var/folders/6v/n7k6518972x6bfvr4s25jp240000gn/T/ipykernel_5096/1296736070.py:110: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  boolArray = self.data[channels] < limit
/var/folders/6v/n7k6518972x6bfvr4s25jp240000gn/T/ipykernel_5096/1296736070.py:115: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq

### Python code for combining two-channel red and green array data with the logarithm (base 2) of the ratio of the red and green channels

In [65]:
# FIX
# for some reason this is only generating a black image....

from numpy import log2

# function for getting the log2 ratio of the channels 
def log2Ratio(data1, data2):
    
    data1 = array(data1) + 1e-3
    data2 = array(data2) + 1e-3
    
    return log2(data1/data2)

imgFile = 'RedGreenArray.png'
rgArray = loadArrayImage(imgFile, 'TwoChannel', 18, 17)

rgArray.combineChannels(0, 1, combFunc=log2Ratio, replace=2)

rgArray.makeImage(20, channels=(2,2,2)).show()  

### Python code for searching each row in a 2D array data to generate a matrix of differences to this row (squared and then square-rooted values are replaced)

In [56]:
# creates a distance matrix from the differences of each row
# adapted method from the hierchaical clustering mehtods
# only creates the distance matrix

imgFile = 'RedGreenArray.png'
rgArray = loadArrayImage(imgFile, 'TwoChannel', 18, 17)
disMatrix = rgArray.distanceMatrix()
disMatrix

array([[0.00000000e+00, 6.36544758e+09, 1.51271695e+10, 3.32158272e+09,
        1.09919084e+10, 1.51928651e+10, 2.21523195e+10, 1.71314757e+10,
        2.77636647e+09, 1.17394949e+08, 5.96021041e+09, 1.51276623e+10,
        3.86348797e+09, 8.22223354e+09, 1.51110649e+10, 2.14586803e+10,
        1.45465933e+10, 2.87317076e+09],
       [6.36544758e+09, 0.00000000e+00, 1.01661972e+10, 5.69572587e+09,
        1.06364731e+10, 1.40070377e+10, 2.19707045e+10, 1.19901940e+10,
        5.43226400e+09, 6.47177176e+09, 8.86528200e+07, 1.02297694e+10,
        5.71490708e+09, 8.23782550e+09, 1.39420844e+10, 2.10204647e+10,
        1.01773661e+10, 5.18086694e+09],
       [1.51271695e+10, 1.01661972e+10, 0.00000000e+00, 1.28670847e+10,
        7.65340349e+09, 1.79723329e+10, 1.75852635e+10, 1.04869309e+10,
        1.20963273e+10, 1.53840694e+10, 1.05887315e+10, 6.65484240e+07,
        1.26714419e+10, 6.82404630e+09, 1.79045631e+10, 1.73254684e+10,
        9.99710952e+09, 1.15679274e+10],
       [3.321

### Python code  to normalize data values for fluorescence intensity data using a log scale

In [61]:
imgFile = 'RedGreenArray.png'
rgArray = loadArrayImage(imgFile, 'TwoChannel', 18, 17)

rgArray.normaliseLogMean()
rgArray.makeImage(25).show()




/var/folders/6v/n7k6518972x6bfvr4s25jp240000gn/T/ipykernel_5096/1296736070.py:104: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  maxVal = self.data[channels].max()
/var/folders/6v/n7k6518972x6bfvr4s25jp240000gn/T/ipykernel_5096/1296736070.py:110: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  boolArray = self.data[channels] < limit
/var/folders/6v/n7k6518972x6bfvr4s25jp240000gn/T/ipykernel_5096/1296736070.py:115: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq